In [6]:
import numpy as np 
import matplotlib.pyplot as plt 
from astropy.table import Table, vstack
import pandas as pd
from astropy.io import ascii
import glob 

In [7]:
pd.set_option('display.max_rows', None)

In [8]:
def import_snana(filename):
    start_data = False
    table = None
    with open(filename) as f:
        for i,line in enumerate(f):
            if 'VARLIST' in line:
                header = line.split()
                header = header[0:9]
                dummy = [['X'*20]]*len(header)
                table = Table(dummy, names=header).copy()[:0]
            elif table is not None:
                data=line.split()
                data=data[0:9]
                if len(data)!=9:
                    break
                table.add_row(data)
    mask = np.array([r['MAGERR'] is not None for r in table])
    table = table[mask]
    table['MJD']    = table['MJD'].astype(float)
    table['FLT'] = table['FLT'].astype(str)
    table['FLUXCAL'] = table['MAGERR'].astype(float)
    table['FLUXCALERR'] = table['MAGERR'].astype(float)
    table['MAG'] = table['MAG'].astype(float)
    table['MAGERR'] = table['MAGERR'].astype(float)
    return(table)

In [9]:
files = glob.glob("*.snana.txt") 
all_data = []
for file in files:
    dataset = import_snana(file)
    all_data.append(dataset)


In [10]:
files

['2019lbi_data.snana.txt',
 '2019ppi_data.snana.txt',
 '2019uez_data.snana.txt',
 '2019wmr_data.snana.txt',
 '2020aafm_data.snana.txt',
 '2020abah_data.snana.txt',
 '2020abjq_data.snana.txt',
 '2020absk_data.snana.txt',
 '2020acun_data.snana.txt',
 '2020aczd_data.snana.txt',
 '2020aeeg_data.snana.txt',
 '2020aeqx_data.snana.txt',
 '2020ama_data.snana.txt',
 '2020awg_data.snana.txt',
 '2020azh_data.snana.txt',
 '2020dbd_data.snana.txt',
 '2020eaf_data.snana.txt',
 '2020ej_data.snana.txt',
 '2020fc_data.snana.txt',
 '2020gcv_data.snana.txt',
 '2020ghq_data.snana.txt',
 '2020hgw_data.snana.txt',
 '2020hrw_data.snana.txt',
 '2020jww_data.snana.txt',
 '2020koc_data.snana.txt',
 '2020kpz_data.snana.txt',
 '2020lfi_data.snana.txt',
 '2020lrr_data.snana.txt',
 '2020mks_data.snana.txt',
 '2020nap_data.snana.txt',
 '2020nim_data.snana.txt',
 '2020ofw_data.snana.txt',
 '2020ovk_data.snana.txt',
 '2020pni_data.snana.txt',
 '2020qmo_data.snana.txt',
 '2020rhg_data.snana.txt',
 '2020rth_data.snana.t

In [11]:
all_df = []
for df in all_data:
    convert = df.to_pandas()
    convert = convert.dropna()
    convert = convert.replace(to_replace = 'g-ZTF', value = 'g')
    convert = convert.replace(to_replace = 'r-ZTF', value = 'r')
    all_df.append(convert)

In [13]:
criteria_met_df = [] 
criteria_met_names = [] 
for df, name in zip(all_df, files):
    peak = df['MAG'].min()
    peak_mjd = df.loc[df['MAG'] == peak, "MJD"].iloc[0]
    before_peak = df[df['MJD'] < peak_mjd ]
    after_peak = df[df['MJD'] >= peak_mjd]
    before_peak_filter_counts = {}
    after_peak_filter_counts = {}
    filters = ['g', 'r', 'i', 'z']
    for filter in filters:
        before_peak_filter_counts[filter] = before_peak[before_peak['FLT'] == filter].shape[0]
        after_peak_filter_counts[filter] = after_peak[after_peak["FLT"] == filter].shape[0]
    if all(n >= 5 for n in before_peak_filter_counts.values()) & all(n >= 10 for n in after_peak_filter_counts.values()):
        criteria_met_df.append(df)
        criteria_met_names.append(name)


In [14]:
criteria_met_names

['2020koc_data.snana.txt',
 '2020kpz_data.snana.txt',
 '2020lrr_data.snana.txt',
 '2020tfb_data.snana.txt',
 '2020tlf_data.snana.txt',
 '2020tly_data.snana.txt',
 '2020tzs_data.snana.txt',
 '2021qes_data.snana.txt',
 '2021sev_data.snana.txt',
 '2021whv_data.snana.txt']

20kpz is not a ii-p, so we will not include it 
20tlf is a flash object, will not be included 
